# 일반음식점.xlsx 전처리
# sheet 1

In [3]:
import pandas as pd
import numpy as np
import math
import datetime

In [5]:
normal_data_origin_1 = pd.read_excel("../data/normal_data_1.xlsx")
normal_data_1 = pd.DataFrame({'인허가일자':normal_data_origin_1['인허가일자'], '폐업일자':normal_data_origin_1['폐업일자'], '소재지전체주소':normal_data_origin_1['소재지전체주소'], '업태구분명':normal_data_origin_1['업태구분명'], 'x':normal_data_origin_1['좌표정보(X)'], 'y':normal_data_origin_1['좌표정보(Y)']})
normal_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   인허가일자    300000 non-null  int64  
 1   폐업일자     214389 non-null  float64
 2   소재지전체주소  299833 non-null  object 
 3   업태구분명    299985 non-null  object 
 4   x        282076 non-null  float64
 5   y        282076 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 13.7+ MB


In [6]:
# '인허가일자'와 '소재지전체주소'에 있는 결측치를 제거
normal_data_1['인허가일자'].dropna()
normal_data_1['소재지전체주소'].dropna()

0                         서울특별시 종로구 신문로1가 163-0번지 
1                            서울특별시 종로구 동숭동 1-49번지 
2                  서울특별시 종로구 수송동 146-1번지 이마빌딩지하1층 
3                           서울특별시 종로구 예지동 151-2번지 
4                    서울특별시 종로구 명륜2가 27-1번지 (지상1층) 
                            ...                   
299995                    서울특별시 영등포구 신길동 410-99번지 
299996                    서울특별시 영등포구 당산동5가 16-7번지 
299997    서울특별시 영등포구 여의도동 14-32번지 정한빌딩    1동  1 2호
299998          서울특별시 영등포구 여의도동 14-11번지 대한빌딩   14호
299999                    서울특별시 영등포구 신길동 1329-0번지 
Name: 소재지전체주소, Length: 299833, dtype: object

In [7]:
# 제거한 후에는 index가 꼬이므로 index 재설정
normal_data_1.reset_index(drop=True, inplace=True)

In [8]:
# 인허가일자 type을 float에서 str로 변환 (19931101.0 > 19931101)
normal_data_1['인허가일자'] = normal_data_1['인허가일자'].astype(str).apply(lambda x: x.replace('.0',""))
normal_data_1['인허가일자']

0         19900428
1         19900810
2         19950720
3         19950722
4         19950516
            ...   
299995    19960913
299996    19871117
299997    19871117
299998    19871118
299999    19871118
Name: 인허가일자, Length: 300000, dtype: object

In [9]:
# 중간중간에 끼어있는 "nan"값을 삭제
normal_data_1 = normal_data_1.drop(normal_data_1[normal_data_1['인허가일자'].map(len)==3].index)

In [10]:
# 삭제 후 인덱스 재정렬
normal_data_1.reset_index(drop=True, inplace=True)

In [11]:
# 에러가 발생하는지 발생하지 않는지 판별하는 for문
c = normal_data_1['인허가일자'].values.tolist()
for i in c:
    if (len(i)!=8 or i=='nan'):
        print(c.index(i),i,"에러 발생")

In [12]:
# 인허가일자 형식을 19931101 > 1993-11-01로 변환
normal_data_1['인허가일자'] = normal_data_1['인허가일자'].apply(lambda x: x[0:4]+"-"+x[4:6]+"-"+x[6:8])
normal_data_1['인허가일자']

0         1990-04-28
1         1990-08-10
2         1995-07-20
3         1995-07-22
4         1995-05-16
             ...    
299995    1996-09-13
299996    1987-11-17
299997    1987-11-17
299998    1987-11-18
299999    1987-11-18
Name: 인허가일자, Length: 300000, dtype: object

In [23]:
normal_data_1.drop(normal_data_1[normal_data_1['인허가일자']=='1675-03-25'],axis=0,inplace=True)

KeyError: '인허가일자'

In [19]:
normal_data_1[normal_data_1['인허가일자']=='1198-12-07']

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y
272895,1198-12-07,20000121.0,서울특별시 구로구 구로동 732-2번지,한식,189716.776523,443071.247638


In [20]:
normal_data_1 = normal_data_1.drop(normal_data_1[normal_data_1['인허가일자']=="1198-12-07"].index)

In [21]:
# datetime형식으로 변환
normal_data_1['인허가일자'] = pd.to_datetime(normal_data_1['인허가일자'])
normal_data_1['인허가일자']

0        1990-04-28
1        1990-08-10
2        1995-07-20
3        1995-07-22
4        1995-05-16
            ...    
299995   1996-09-13
299996   1987-11-17
299997   1987-11-17
299998   1987-11-18
299999   1987-11-18
Name: 인허가일자, Length: 299997, dtype: datetime64[ns]

In [23]:
# 인허가일자를 index로 만들기(인허가일자가 index가 되어야 날짜별로 slicing이 가능해짐)
normal_data_1 = normal_data_1.set_index(normal_data_1['인허가일자'], drop=False, append=False, inplace=False)
# inplace=False 할 경우 다른 변수에 저장해야 값이 보존됨

In [24]:
# 2015년부터 2020년 1분기까지의 데이터만 남겨놓기
normal_data_1_15_20 = normal_data_1['2015-01-01':'2020-03-31']
normal_data_1_15_20.to_excel("normal_data_1_15_20.xlsx")

In [25]:
normal_data_1_15_20.head(10)

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y
인허가일자,,,,,,
2019-10-16,2019-10-16,NaN,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483
2019-10-16,2019-10-16,NaN,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407
2019-10-16,2019-10-16,NaN,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743
2019-11-25,2019-11-25,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434
2019-11-25,2019-11-25,NaN,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765
2019-11-25,2019-11-25,NaN,서울특별시 종로구 종로1가 24번지 르메이에르종로타운1,경양식,198150.300374,452019.212643
2019-11-14,2019-11-14,NaN,서울특별시 종로구 소격동 88-17번지,경양식,198250.951353,453098.366692
2019-11-14,2019-11-14,NaN,서울특별시 종로구 관훈동 155-2번지,한식,198498.918633,452473.403750
2019-12-02,2019-12-02,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,"외국음식전문점(인도,태국등)",198984.839771,452270.384434


In [26]:
normal_data_1_15_20.reset_index(drop=True, inplace=False)

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y
0,2019-10-16,NaN,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483
1,2019-10-16,NaN,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407
2,2019-10-16,NaN,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743
3,2019-11-25,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434
4,2019-11-25,NaN,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765
...,...,...,...,...,...,...
42896,2018-09-28,20200122.0,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634
42897,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229
42898,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229
42899,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 5층,뷔페식,191400.318861,442715.143229


In [27]:
# 소재지전체주소가 nan값인지 아닌지 판별하는 함수 (nan값이면 해당 값 대신에 true를 넣는다.)
def isnan_not(x):
    for i in x:
        try:
            x[x.index(i)] = math.isnan(i)
        except TypeError:
            continue

In [29]:
alpha = normal_data_1_15_20['소재지전체주소'].values.tolist()
isnan_not(alpha)

In [31]:
normal_data_1_15_20['소재지전체주소_new'] = alpha

<ipython-input-31-22caa9f1f6d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data_1_15_20['소재지전체주소_new'] = alpha


In [32]:
normal_data_1_15_20

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,소재지전체주소_new
인허가일자,,,,,,,
2019-10-16,2019-10-16,NaN,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483,서울특별시 종로구 익선동 26-1번지
2019-10-16,2019-10-16,NaN,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407,서울특별시 종로구 옥인동 107번지
2019-10-16,2019-10-16,NaN,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743,서울특별시 종로구 청진동 128 Tower 8
2019-11-25,2019-11-25,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434,서울특별시 종로구 낙원동 110번지 경한빌딩
2019-11-25,2019-11-25,NaN,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765,서울특별시 종로구 창신동 580-25번지
...,...,...,...,...,...,...,...
2018-09-28,2018-09-28,20200122.0,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층


In [34]:
normal_data_1_15_20[normal_data_1_15_20['소재지전체주소_new']==True]

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,소재지전체주소_new
인허가일자,,,,,,,
2017-07-11,2017-07-11,NaN,NaN,분식,196992.000000,451805.000000,True
2019-02-19,2019-02-19,NaN,NaN,호프/통닭,196173.938452,456154.897670,True
2015-03-30,2015-03-30,NaN,NaN,기타,201168.350187,453134.920223,True
2016-02-15,2016-02-15,NaN,NaN,중국식,200054.536347,453799.405798,True
2015-09-16,2015-09-16,NaN,NaN,경양식,197764.000000,452030.000000,True
...,...,...,...,...,...,...,...
2015-02-09,2015-02-09,NaN,NaN,호프/통닭,190691.303227,448231.307431,True
2015-02-11,2015-02-11,NaN,NaN,한식,191444.655225,447251.740221,True
2016-04-14,2016-04-14,20181217.0,NaN,호프/통닭,192747.419300,447054.988337,True


In [35]:
# 소재지전체주소_new가 True인 값(nan값인 경우) 삭제한다.
normal_data_1_15_20 = normal_data_1_15_20.drop(normal_data_1_15_20[normal_data_1_15_20['소재지전체주소_new']==True].index)
normal_data_1_15_20[normal_data_1_15_20['소재지전체주소_new']==True]

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,소재지전체주소_new
인허가일자,,,,,,,


In [37]:
normal_data_1_15_20.drop(['소재지전체주소_new'],axis=1)

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y
인허가일자,,,,,,
2019-10-16,2019-10-16,NaN,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483
2019-10-16,2019-10-16,NaN,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407
2019-10-16,2019-10-16,NaN,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743
2019-11-25,2019-11-25,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434
2019-11-25,2019-11-25,NaN,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765
...,...,...,...,...,...,...
2018-09-28,2018-09-28,20200122.0,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229


In [38]:
beta = normal_data_1_15_20['소재지전체주소'].values.tolist()
for i in beta:
    if i.split(" ")[0]!="서울특별시":
        beta[beta.index(i)] = True
    else:
        beta[beta.index(i)] = False
print(beta)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

In [40]:
normal_data_1_15_20['notseoul'] = beta
normal_data_1_15_20.head(10)

<ipython-input-40-fed7a1f148e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data_1_15_20['notseoul'] = beta


,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,소재지전체주소_new,notseoul
인허가일자,,,,,,,,
2019-10-16,2019-10-16,NaN,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483,서울특별시 종로구 익선동 26-1번지,False
2019-10-16,2019-10-16,NaN,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407,서울특별시 종로구 옥인동 107번지,False
2019-10-16,2019-10-16,NaN,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743,서울특별시 종로구 청진동 128 Tower 8,False
2019-11-25,2019-11-25,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434,서울특별시 종로구 낙원동 110번지 경한빌딩,False
2019-11-25,2019-11-25,NaN,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765,서울특별시 종로구 창신동 580-25번지,False
2019-11-25,2019-11-25,NaN,서울특별시 종로구 종로1가 24번지 르메이에르종로타운1,경양식,198150.300374,452019.212643,서울특별시 종로구 종로1가 24번지 르메이에르종로타운1,False
2019-11-14,2019-11-14,NaN,서울특별시 종로구 소격동 88-17번지,경양식,198250.951353,453098.366692,서울특별시 종로구 소격동 88-17번지,False
2019-11-14,2019-11-14,NaN,서울특별시 종로구 관훈동 155-2번지,한식,198498.918633,452473.403750,서울특별시 종로구 관훈동 155-2번지,False
2019-12-02,2019-12-02,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,"외국음식전문점(인도,태국등)",198984.839771,452270.384434,서울특별시 종로구 낙원동 110번지 경한빌딩,False


In [41]:
normal_data_1_15_20.drop(normal_data_1_15_20[normal_data_1_15_20['notseoul']==True].index)

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,소재지전체주소_new,notseoul
인허가일자,,,,,,,,
2019-10-16,2019-10-16,NaN,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483,서울특별시 종로구 익선동 26-1번지,False
2019-10-16,2019-10-16,NaN,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407,서울특별시 종로구 옥인동 107번지,False
2019-10-16,2019-10-16,NaN,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743,서울특별시 종로구 청진동 128 Tower 8,False
2019-11-25,2019-11-25,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434,서울특별시 종로구 낙원동 110번지 경한빌딩,False
2019-11-25,2019-11-25,NaN,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765,서울특별시 종로구 창신동 580-25번지,False
...,...,...,...,...,...,...,...,...
2018-09-28,2018-09-28,20200122.0,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,False
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,False
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,False


In [43]:
normal_data_1_15_20.drop(['소재지전체주소_new', 'notseoul'],axis=1,inplace=True)

C:\Users\dpffp\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [45]:
normal_data_1_15_20

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y
인허가일자,,,,,,
2019-10-16,2019-10-16,NaN,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483
2019-10-16,2019-10-16,NaN,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407
2019-10-16,2019-10-16,NaN,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743
2019-11-25,2019-11-25,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434
2019-11-25,2019-11-25,NaN,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765
...,...,...,...,...,...,...
2018-09-28,2018-09-28,20200122.0,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229


In [46]:
normal_data_1_15_20.reset_index(drop=True, inplace=False)

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y
0,2019-10-16,NaN,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483
1,2019-10-16,NaN,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407
2,2019-10-16,NaN,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743
3,2019-11-25,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434
4,2019-11-25,NaN,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765
...,...,...,...,...,...,...
40408,2018-09-28,20200122.0,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634
40409,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229
40410,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229
40411,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 5층,뷔페식,191400.318861,442715.143229


In [48]:
normal_data_1_15_20['구'] = normal_data_1_15_20['소재지전체주소'].apply(lambda x: x.split(" ")[1])

<ipython-input-48-4b064209f136>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data_1_15_20['구'] = normal_data_1_15_20['소재지전체주소'].apply(lambda x: x.split(" ")[1])


In [49]:
normal_data_1_15_20['동'] = normal_data_1_15_20['소재지전체주소'].apply(lambda x: x.split(" ")[2])

<ipython-input-49-b3cc18920f48>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data_1_15_20['동'] = normal_data_1_15_20['소재지전체주소'].apply(lambda x: x.split(" ")[2])


In [50]:
omega = normal_data_1_15_20['동'].values
empty = []
for i in omega:
    if (i[-1]=="가" or i[-1]=="로"):
        empty.append(i)

In [51]:
empty_set = set(empty)
print(empty_set) # unique한 set을 찾기

{'용산동2가', '동소문동6가', '양평동2가', '당산동1가', '청파동2가', '저동2가', '안암동2가', '명륜4가', '당산동6가', '양평동4가', '을지로3가', '보문동2가', '문래동6가', '동선동5가', '성수동2가', '삼선동3가', '금호동3가', '충무로4가', '만리동2가', '인현동1가', '필동1가', '종로2가', '충무로1가', '회현동2가', '보문동5가', '금호동2가', '동선동2가', '당산동2가', '당산동4가', '태평로2가', '한강로1가', '문래동3가', '만리동1가', '남대문로3가', '삼선동2가', '영등포동5가', '충정로1가', '남산동2가', '원효로4가', '종로5가', '세종로', '남산동3가', '봉래동1가', '문래동4가', '원효로1가', '삼선동4가', '영등포동6가', '성수동1가', '동소문동7가', '동소문동4가', '삼선동1가', '문래동1가', '양평동1가', '명륜1가', '안암동4가', '장충동2가', '한강로3가', '신문로1가', '신문로2가', '영등포동4가', '양평동3가', '을지로2가', '안암동5가', '동선동4가', '영등포동7가', '동소문동2가', '충무로2가', '동선동1가', '의주로1가', '태평로1가', '충정로2가', '저동1가', '용산동3가', '문래동5가', '명륜3가', '안암동3가', '동선동3가', '을지로5가', '명동1가', '보문동4가', '원효로2가', '당산동5가', '광희동1가', '을지로4가', '삼선동5가', '종로6가', '금호동1가', '필동2가', '종로1가', '성북동1가', '충정로3가', '동소문동1가', '충무로3가', '봉래동2가', '필동3가', '을지로6가', '문래동2가', '용산동1가', '보문동6가', '남대문로2가', '광희동2가', '보문동7가', '남대문로4가', '당산동3가', '영등포동1가', '동소문동5가', '회현동3가', '영등포동2가', '안암동1가', '명동2가', '용산동6가', '

In [53]:
dong = normal_data_1_15_20['동'].values.tolist()

In [54]:
for i in dong:
    if (i in ["종로5가", "종로6가"]):
        dong[dong.index(i)]="종로5.6가동"
    elif (i in ["통의동", "적선동", "체부동", "필운동", "내자동", "사직동", "도렴동", "당주동", "내수동", "세종로", "신문로1가", "신문로2가"]):
        dong[dong.index(i)]="사직동"
    elif (i in ['명륜1가', '명륜2가', '명륜3가', '명륜4가']):
        dong[dong.index(i)]="혜화동"
    elif (i in ['원효로1가', '원효로2가', '문배동', '신계동']):
        dong[dong.index(i)]="원효로1동"
    elif (i in ['신창동', '산천동', '청암동', '원효로3가', '원효로4가']):
        dong[dong.index(i)]="원효로2동"
    elif (i in ['당산동1가', '당산동2가', '당산동3가']):
        dong[dong.index(i)]="당산1동"
    elif (i in ['당산동4가', '당산동5가', '당산동6가', '당산동']):
        dong[dong.index(i)]="당산2동"
    elif (i in ['돈암동', '동소문동6가', '동소문동7가', '동선동1가', '동선동2가', '동선동3가', '동선동4가', '동선동5가']):
        dong[dong.index(i)]="동선동"
    elif (i in ['남창동', '남대문로3가', '남대문로4가', '남대문로5가', '봉래동1가', '봉래동2가', '회현동1가', '회현동2가', '충무로1가', '순화동']):
        dong[dong.index(i)]="회현동"
    elif (i in ['무교동', '다동', '태평로1가', '을지로1가', '을지로2가', '남대문로1가', '삼각동', '수하동', '장교동', '수표동', '남대문로2가', '회현동1가', '회현동2가', '회현동3가', '충무로1가', '충무로2가', '명동1가', '명동2가', '남산동1가', '남산동2가', '남산동3가', '저동1가', '예장동']):
        dong[dong.index(i)]="명동"
    elif (i in ['한강로1가', '한강로2가', '용산동3가', '용산동5가', '한강로3가']):
        dong[dong.index(i)]="한강로동"
    elif (i in ['을지로4가', '을지로5가', '주교동', '방산동', '을지로3가', '입정동', '산림동', '초동', '인현동1가', '저동2가']):
        dong[dong.index(i)]="을지로동"
    elif (i in ['돈암동', '동소문동2가', '동소문동3가', '동소문동5가', '삼선동1가', '삼선동2가', '삼선동3가', '삼선동4가', '삼선동5가']):
        dong[dong.index(i)]="삼선동"
    elif (i in ['세종로', '청진동', '서린동', '수송동', '중학동', '종로1가', '공평동', '관훈동', '견지동', '와룡동', '권농동', '운니동', '익선동', '경운동', '관철동', '인사동', '낙원동', '종로2가', '훈정동', '묘동', '봉익동', '돈의동', '장사동', '관수동', '종로3가', '인의동', '예지동', '원남동', '종로4가']):
        dong[dong.index(i)]="종로1.2.3.4가동"
    elif (i in ['보문동4가', '보문동5가', '보문동6가', '보문동7가', '보문동1가', '보문동2가', '보문동3가']):
        dong[dong.index(i)]="보문동"
    elif (i in ['금호동4가']):
        dong[dong.index(i)]="금호4가동"
    elif (i in ['금호동2가', '금호동3가']):
        dong[dong.index(i)]="금호2.3가동"
    elif (i in ['금호동1가']):
        dong[dong.index(i)]="금호1가동"
    elif (i in ['영등포동1가', '영등포동2가', '영등포동3가', '영등포동4가', '영등포동5가', '영등포동6가', '영등포동7가', '영등포동8가']):
        dong[dong.index(i)]="영등포동"
    elif (i in ['영등포동', '신길동']):
        dong[dong.index(i)]="영등포본동"
    elif (i in ['충무로4가', '충무로5가', '묵정동', '필동1가', '필동2가', '필동3가', '남학동', '주자동', '예장동', '장충동2가', '충무로3가']):
        dong[dong.index(i)]="필동"
    elif (i in ['소공동', '북창동', '태평로2가', '남대문로2가', '남대문로3가', '남대문로4가', '봉래동1가', '서소문동', '정동', '순화동', '의주로1가', '충정로1가']):
        dong[dong.index(i)]="소공동"
    elif (i in ['충무로4가', '충무로5가', '인현동2가', '예관동', '광희동1가', '광희동2가', '쌍림동', '을지로6가', '을지로7가', '오장동']):
        dong[dong.index(i)]="광희동"
    elif (i in ['안암동1가', '안암동2가', '안암동3가', '안암동4가', '안암동5가']):
        dong[dong.index(i)]="안암동"
    elif (i in ['서계동', '청파동1가', '청파동2가', '청파동3가']):
        dong[dong.index(i)]="청파동"
    elif (i in ['양평동1가', '양평동2가', '양평동3가']):
        dong[dong.index(i)]="양평1동"
    elif (i in ['양평동3가', '양평동4가', '양평동5가', '양평동6가', '양화동', '양평동']):
        dong[dong.index(i)]="양평2동"
    elif (i in ['용산동2가', '용산동4가']):
        dong[dong.index(i)]="용산2가동"
    elif (i in ['성수동1가']):
        dong[dong.index(i)]="성수1가1동"
    elif (i in ['성수동2가']):
        dong[dong.index(i)]="성수2가1동"
    elif (i in ['성북동', '성북동1가', '동소문동1가', '동소문동4가']):
        dong[dong.index(i)]="성북동"
    elif (i in ['문래동1가', '문래동2가', '문래동3가', '문래동4가', '문래동5가', '문래동6가']):
        dong[dong.index(i)]="문래동"
    elif (i in ['원효로1가', '원효로2가', '문배동', '신계동']):
        dong[dong.index(i)]="원효로1동"
    elif (i in ['신창동', '산천동', '청암동', '원효로3가', '원효로4가']):
        dong[dong.index(i)]="원효로2동"
    elif (i in ['중림동', '의주로2가', '만리동1가', '만리동2가']):
        dong[dong.index(i)]="중림동"
    elif (i in ['충정로2가', '충정로3가', '합동', '미근동', '북아현동']):
        dong[dong.index(i)]="충현동"
    elif (i in ['묵정동', '장충동1가', '장충동2가']):
        dong[dong.index(i)]="장충동"
    elif (i in ['동빙고동', '서빙고동', '주성동', '용산동6가']):
        dong[dong.index(i)]="서빙고동"
    elif (i in ['갈월동', '남영동', '용산동1가', '동자동']):
        dong[dong.index(i)]="남영동"
print(dong)

['종로1.2.3.4가동', '옥인동', '종로1.2.3.4가동', '종로1.2.3.4가동', '창신동', '종로1.2.3.4가동', '소격동', '종로1.2.3.4가동', '종로1.2.3.4가동', '사직동', '종로1.2.3.4가동', '사직동', '종로1.2.3.4가동', '종로1.2.3.4가동', '종로5.6가동', '누하동', '동숭동', '창신동', '종로1.2.3.4가동', '종로1.2.3.4가동', '동숭동', '종로1.2.3.4가동', '효제동', '종로1.2.3.4가동', '종로5.6가동', '이화동', '누하동', '충신동', '안국동', '사직동', '종로1.2.3.4가동', '종로1.2.3.4가동', '종로1.2.3.4가동', '종로1.2.3.4가동', '사직동', '효제동', '종로1.2.3.4가동', '종로1.2.3.4가동', '종로1.2.3.4가동', '사직동', '종로1.2.3.4가동', '종로1.2.3.4가동', '사직동', '종로1.2.3.4가동', '종로1.2.3.4가동', '가회동', '연건동', '구기동', '종로1.2.3.4가동', '궁정동', '종로1.2.3.4가동', '옥인동', '동숭동', '종로1.2.3.4가동', '사직동', '혜화동', '신영동', '효자동', '동숭동', '종로1.2.3.4가동', '종로1.2.3.4가동', '행촌동', '사직동', '숭인동', '안국동', '사직동', '홍지동', '창신동', '소격동', '혜화동', '사직동', '종로1.2.3.4가동', '종로1.2.3.4가동', '혜화동', '종로1.2.3.4가동', '종로1.2.3.4가동', '부암동', '평동', '무악동', '종로1.2.3.4가동', '종로1.2.3.4가동', '종로1.2.3.4가동', '삼청동', '재동', '사직동', '창신동', '사직동', '무악동', '효자동', '혜화동', '사직동', '숭인동', '사직동', '홍지동', '종로1.2.3.4가동', '사직동', '종로1.2.3.4가동', '사간동', '부암

In [55]:
# 돌리고 나서 빼먹은 부분이 있는지 찾기
for i in dong:
    if (i[-1]=="가" or i[-1]=="로"):
        print(i)

In [56]:
normal_data_1_15_20['동_new'] = dong
normal_data_1_15_20

<ipython-input-56-3e72ddef8fa0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data_1_15_20['동_new'] = dong


,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,구,동,동_new
인허가일자,,,,,,,,,
2019-10-16,2019-10-16,NaN,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483,종로구,익선동,종로1.2.3.4가동
2019-10-16,2019-10-16,NaN,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407,종로구,옥인동,옥인동
2019-10-16,2019-10-16,NaN,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743,종로구,청진동,종로1.2.3.4가동
2019-11-25,2019-11-25,NaN,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434,종로구,낙원동,종로1.2.3.4가동
2019-11-25,2019-11-25,NaN,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765,종로구,창신동,창신동
...,...,...,...,...,...,...,...,...,...
2018-09-28,2018-09-28,20200122.0,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634,영등포구,양평동4가,양평2동
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229,영등포구,대림동,대림동
2018-09-13,2018-09-13,20200609.0,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229,영등포구,대림동,대림동


In [57]:
normal_data_1_15_20['폐업일자'] = normal_data_1_15_20['폐업일자'].astype(str).apply(lambda x: x.replace('.0',""))
normal_data_1_15_20

<ipython-input-57-9f823b1c708b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data_1_15_20['폐업일자'] = normal_data_1_15_20['폐업일자'].astype(str).apply(lambda x: x.replace('.0',""))


,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,구,동,동_new
인허가일자,,,,,,,,,
2019-10-16,2019-10-16,nan,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483,종로구,익선동,종로1.2.3.4가동
2019-10-16,2019-10-16,nan,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407,종로구,옥인동,옥인동
2019-10-16,2019-10-16,nan,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743,종로구,청진동,종로1.2.3.4가동
2019-11-25,2019-11-25,nan,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434,종로구,낙원동,종로1.2.3.4가동
2019-11-25,2019-11-25,nan,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765,종로구,창신동,창신동
...,...,...,...,...,...,...,...,...,...
2018-09-28,2018-09-28,20200122,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634,영등포구,양평동4가,양평2동
2018-09-13,2018-09-13,20200609,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229,영등포구,대림동,대림동
2018-09-13,2018-09-13,20200609,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229,영등포구,대림동,대림동


In [59]:
closedstore = normal_data_1_15_20['폐업일자'].values.tolist()
for i in closedstore:
    if i=="nan":
        continue
    else:
        closedstore[closedstore.index(i)] = pd.to_datetime(i[0:4]+"-"+i[4:6]+"-"+i[6:8])
print(closedstore)

['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'

In [61]:
normal_data_1_15_20['폐업일자'] = closedstore
normal_data_1_15_20

<ipython-input-61-034905363aea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data_1_15_20['폐업일자'] = closedstore


,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,구,동,동_new
인허가일자,,,,,,,,,
2019-10-16,2019-10-16,nan,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483,종로구,익선동,종로1.2.3.4가동
2019-10-16,2019-10-16,nan,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407,종로구,옥인동,옥인동
2019-10-16,2019-10-16,nan,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743,종로구,청진동,종로1.2.3.4가동
2019-11-25,2019-11-25,nan,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434,종로구,낙원동,종로1.2.3.4가동
2019-11-25,2019-11-25,nan,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765,종로구,창신동,창신동
...,...,...,...,...,...,...,...,...,...
2018-09-28,2018-09-28,2020-01-22 00:00:00,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634,영등포구,양평동4가,양평2동
2018-09-13,2018-09-13,2020-06-09 00:00:00,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229,영등포구,대림동,대림동
2018-09-13,2018-09-13,2020-06-09 00:00:00,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229,영등포구,대림동,대림동


In [62]:
normal_data_1_15_20['인허가일자'] = pd.to_datetime(normal_data_1_15_20['인허가일자'])
normal_data_1_15_20

<ipython-input-62-67fa8465bed3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data_1_15_20['인허가일자'] = pd.to_datetime(normal_data_1_15_20['인허가일자'])


,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,구,동,동_new
인허가일자,,,,,,,,,
2019-10-16,2019-10-16,nan,서울특별시 종로구 익선동 26-1번지,한식,199013.621162,452566.572483,종로구,익선동,종로1.2.3.4가동
2019-10-16,2019-10-16,nan,서울특별시 종로구 옥인동 107번지,한식,197102.594677,453154.492407,종로구,옥인동,옥인동
2019-10-16,2019-10-16,nan,서울특별시 종로구 청진동 128 Tower 8,기타,198213.955223,452045.296743,종로구,청진동,종로1.2.3.4가동
2019-11-25,2019-11-25,nan,서울특별시 종로구 낙원동 110번지 경한빌딩,한식,198984.839771,452270.384434,종로구,낙원동,종로1.2.3.4가동
2019-11-25,2019-11-25,nan,서울특별시 종로구 창신동 580-25번지,"외국음식전문점(인도,태국등)",200941.928779,452217.462765,종로구,창신동,창신동
...,...,...,...,...,...,...,...,...,...
2018-09-28,2018-09-28,2020-01-22 00:00:00,서울특별시 영등포구 양평동4가 51-1번지 경남아너스빌 상가에이동 1층 108호,일식,190757.977577,448162.410634,영등포구,양평동4가,양평2동
2018-09-13,2018-09-13,2020-06-09 00:00:00,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 3층,뷔페식,191400.318861,442715.143229,영등포구,대림동,대림동
2018-09-13,2018-09-13,2020-06-09 00:00:00,서울특별시 영등포구 대림동 993-15번지 대림동웨딩홀 4층,뷔페식,191400.318861,442715.143229,영등포구,대림동,대림동


In [63]:
normal_data_1_openrate = pd.DataFrame({'인허가일자':normal_data_1_15_20['인허가일자'], '구':normal_data_1_15_20['구'], '동':normal_data_1_15_20['동_new'], 'x':normal_data_1_15_20['x'],'y':normal_data_1_15_20['y']})

In [64]:
def changeyear_opened():
    for i in [2015, 2016, 2017, 2018, 2019, 2020]:
        cnt_1 = []
        openday = normal_data_1_openrate['인허가일자'].values.tolist()
        for j in openday:
            if (pd.to_datetime(j) >= datetime.datetime(i,1,1) and pd.to_datetime(j) <= datetime.datetime(i,3,31)):
                cnt_1.append(1)
            else:
                cnt_1.append(0)
        normal_data_1_openrate[str(i)+"_1"] = cnt_1

        cnt_2 = []
        openday = normal_data_1_openrate['인허가일자'].values.tolist()
        for j in openday:
            if (pd.to_datetime(j) >= datetime.datetime(i,4,1) and pd.to_datetime(j) <= datetime.datetime(i,6,30)):
                cnt_2.append(1)
            else:
                cnt_2.append(0)
        normal_data_1_openrate[str(i)+"_2"] = cnt_2

        cnt_3 = []
        openday = normal_data_1_openrate['인허가일자'].values.tolist()
        for j in openday:
            if (pd.to_datetime(j) >= datetime.datetime(i,7,1) and pd.to_datetime(j) <= datetime.datetime(i,9,30)):
                cnt_3.append(1)
            else:
                cnt_3.append(0)
        normal_data_1_openrate[str(i)+"_3"] = cnt_3

        cnt_4 = []
        openday = normal_data_1_openrate['인허가일자'].values.tolist()
        for j in openday:
            if (pd.to_datetime(j) >= datetime.datetime(i,10,1) and pd.to_datetime(j) <= datetime.datetime(i,12,31)):
                cnt_4.append(1)
            else:
                cnt_4.append(0)
        normal_data_1_openrate[str(i)+"_4"] = cnt_4

In [65]:
changeyear_opened()

In [66]:
normal_data_1_openrate.drop(['2020_2','2020_3','2020_4'], axis=1, inplace=True)

,인허가일자,구,동,x,y,2015_1,2015_2,2015_3,2015_4,2016_1,...,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1
인허가일자,,,,,,,,,,,,,,,,,,,,,
2019-10-16,2019-10-16,종로구,종로1.2.3.4가동,199013.621162,452566.572483,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2019-10-16,2019-10-16,종로구,옥인동,197102.594677,453154.492407,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2019-10-16,2019-10-16,종로구,종로1.2.3.4가동,198213.955223,452045.296743,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2019-11-25,2019-11-25,종로구,종로1.2.3.4가동,198984.839771,452270.384434,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2019-11-25,2019-11-25,종로구,창신동,200941.928779,452217.462765,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-09-28,2018-09-28,영등포구,양평2동,190757.977577,448162.410634,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2018-09-13,2018-09-13,영등포구,대림동,191400.318861,442715.143229,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2018-09-13,2018-09-13,영등포구,대림동,191400.318861,442715.143229,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [75]:
normal_data_1_openrate.reset_index(drop=True, inplace=True)

In [76]:
normal_data_1_openrate

,인허가일자,구,동,x,y,2015_1,2015_2,2015_3,2015_4,2016_1,...,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1
0,2019-10-16,종로구,종로1.2.3.4가동,199013.621162,452566.572483,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2019-10-16,종로구,옥인동,197102.594677,453154.492407,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2019-10-16,종로구,종로1.2.3.4가동,198213.955223,452045.296743,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2019-11-25,종로구,종로1.2.3.4가동,198984.839771,452270.384434,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2019-11-25,종로구,창신동,200941.928779,452217.462765,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40408,2018-09-28,영등포구,양평2동,190757.977577,448162.410634,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
40409,2018-09-13,영등포구,대림동,191400.318861,442715.143229,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
40410,2018-09-13,영등포구,대림동,191400.318861,442715.143229,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
40411,2018-09-13,영등포구,대림동,191400.318861,442715.143229,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [77]:
normal_data_1_openrate.to_excel("normal_data_1_openrate.xlsx")

In [85]:
normal_data_1_closedrate = pd.DataFrame({'폐업일자':normal_data_1_15_20['폐업일자'], '구':normal_data_1_15_20['구'], '동':normal_data_1_15_20['동_new'], 'x':normal_data_1_15_20['x'],'y':normal_data_1_15_20['y']})

In [86]:
normal_data_1_closedrate

,폐업일자,구,동,x,y
0,nan,종로구,종로1.2.3.4가동,199013.621162,452566.572483
1,nan,종로구,옥인동,197102.594677,453154.492407
2,nan,종로구,종로1.2.3.4가동,198213.955223,452045.296743
3,nan,종로구,종로1.2.3.4가동,198984.839771,452270.384434
4,nan,종로구,창신동,200941.928779,452217.462765
...,...,...,...,...,...
40408,2020-01-22 00:00:00,영등포구,양평2동,190757.977577,448162.410634
40409,2020-06-09 00:00:00,영등포구,대림동,191400.318861,442715.143229
40410,2020-06-09 00:00:00,영등포구,대림동,191400.318861,442715.143229
40411,2020-06-09 00:00:00,영등포구,대림동,191400.318861,442715.143229


In [87]:
def changeyear_closed():
    for i in [2015, 2016, 2017, 2018, 2019, 2020]:
        cnt_1 = []
        closedday = normal_data_1_closedrate['폐업일자'].values.tolist()
        for j in closedday:
            if (pd.to_datetime(j) >= datetime.datetime(i,1,1) and pd.to_datetime(j) <= datetime.datetime(i,3,31)):
                cnt_1.append(1)
            elif (str(j) == "nan"):
                cnt_1.append(0)
            else:
                cnt_1.append(0)
        normal_data_1_closedrate[str(i)+"_1"] = cnt_1

        cnt_2 = []
        closedday = normal_data_1_closedrate['폐업일자'].values.tolist()
        for j in closedday:
            if (pd.to_datetime(j) >= datetime.datetime(i,4,1) and pd.to_datetime(j) <= datetime.datetime(i,6,30)):
                cnt_2.append(1)
            elif (str(j) == "nan"):
                cnt_2.append(0)
            else:
                cnt_2.append(0)
        normal_data_1_closedrate[str(i)+"_2"] = cnt_2

        cnt_3 = []
        closedday = normal_data_1_closedrate['폐업일자'].values.tolist()
        for j in closedday:
            if (pd.to_datetime(j) >= datetime.datetime(i,7,1) and pd.to_datetime(j) <= datetime.datetime(i,9,30)):
                cnt_3.append(1)
            elif (str(j) == "nan"):
                cnt_3.append(0)
            else:
                cnt_3.append(0)
        normal_data_1_closedrate[str(i)+"_3"] = cnt_3

        cnt_4 = []
        closedday = normal_data_1_closedrate['폐업일자'].values.tolist()
        for j in closedday:
            if (pd.to_datetime(j) >= datetime.datetime(i,10,1) and pd.to_datetime(j) <= datetime.datetime(i,12,31)):
                cnt_4.append(1)
            elif (str(j) == "nan"):
                cnt_4.append(0)
            else:
                cnt_4.append(0)
        normal_data_1_closedrate[str(i)+"_4"] = cnt_4

In [88]:
changeyear_closed()

In [89]:
normal_data_1_closedrate.drop(['2020_2','2020_3','2020_4'], axis=1, inplace=True)
normal_data_1_closedrate
normal_data_1_closedrate.to_excel("normal_data_1_closedrate.xlsx")